In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Zahinos\AppData\Local\Temp\ipykernel_14872\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import re

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [13]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\Zahinos\Desktop\AI\Week4\Day3\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')
data = data.head(1000)
data.fillna("", inplace=True)

print(data.shape)
print(data.columns)

(1000, 2)
Index(['text', 'label'], dtype='object')


### Let's divide the training and test set into two partitions

In [14]:
from sklearn.model_selection import train_test_split

X = data["text"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])

Training set size: 800
Test set size: 200


## Data Preprocessing

In [15]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [16]:
# Your code
def clean1(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<[^>]+>', '', text)
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [17]:
# Your code
def clean2(text):
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [18]:
# Your code
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stopwords.words("english")]
    return ' '.join(filtered)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [19]:
# Your code
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zahinos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Zahinos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
data['text'] = data['text'].astype(str)
data['text'] = data['text'].apply(clean1).apply(clean2).apply(remove_stopwords).apply(lemmatize_text)
data['label'] = data['label'].astype(str).str.strip().str.lower()
data = data[data['text'].str.len() > 0]

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

ham_texts = data[data['label'] == 'ham']['text']
spam_texts = data[data['label'] == 'spam']['text']

ham_texts = ham_texts[ham_texts.str.len() > 0]
spam_texts = spam_texts[spam_texts.str.len() > 0]

if ham_texts.empty or spam_texts.empty:
    print("No hay datos válidos en ham o spam.")
else:
    vectorizer_ham = CountVectorizer()
    vectorizer_spam = CountVectorizer()

    ham_bow = vectorizer_ham.fit_transform(ham_texts)
    spam_bow = vectorizer_spam.fit_transform(spam_texts)

    ham_word_counts = ham_bow.sum(axis=0).A1
    spam_word_counts = spam_bow.sum(axis=0).A1

    ham_vocab = vectorizer_ham.get_feature_names_out()
    spam_vocab = vectorizer_spam.get_feature_names_out()

    ham_freq = dict(zip(ham_vocab, ham_word_counts))
    spam_freq = dict(zip(spam_vocab, spam_word_counts))

    print("🔹 Top 10 palabras en HAM:")
    print(sorted(ham_freq.items(), key=lambda x: x[1], reverse=True)[:10])

    print("\n🔸 Top 10 palabras en SPAM:")
    print(sorted(spam_freq.items(), key=lambda x: x[1], reverse=True)[:10])

No hay datos válidos en ham o spam.


## Extra features

In [23]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
data['preprocessed_text'] = data['text']
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join([
    "free", "cheap", "sex", "money", "account", "bank",
    "fund", "transfer", "transaction", "win", "deposit", "password"
])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list, case=False, regex=True) * 1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words, case=False, regex=True) * 1
data_train['text_len'] = data_train['preprocessed_text'].apply(len)

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list, case=False, regex=True) * 1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words, case=False, regex=True) * 1
data_val['text_len'] = data_val['preprocessed_text'].apply(len)

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
258,preference front back end set pm,0,preference front back end set pm,1,0,32
960,pls print monday february jilotylc state gov e...,0,pls print monday february jilotylc state gov e...,1,0,61
130,released,0,released,1,0,8
581,expect immediate response call also let direct...,1,expect immediate response call also let direct...,1,0,213
467,republic way homeimportance love th cg,0,republic way homeimportance love th cg,1,0,38


## How would work the Bag of Words with Count Vectorizer concept?

In [24]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["I love NLP", "NLP is fun", "I love fun"]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out())
print(X.toarray())

['fun' 'is' 'love' 'nlp']
[[0 0 1 1]
 [1 1 0 1]
 [1 0 1 0]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(data['text'])

print("Forma del dataset vectorizado con TF-IDF:", X_tfidf.shape)

Forma del dataset vectorizado con TF-IDF: (983, 29931)


## And the Train a Classifier?

In [27]:
# Your code
from sklearn.model_selection import train_test_split

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(
    X_tfidf, data['label'], test_size=0.2, random_state=42, stratify=data['label']
)

clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.934010152284264

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.94       109
           1       0.88      0.99      0.93        88

    accuracy                           0.93       197
   macro avg       0.93      0.94      0.93       197
weighted avg       0.94      0.93      0.93       197



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [28]:
# Your code
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Vectorización
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')  # ← cambia esta línea según el caso
X_vect = vectorizer.fit_transform(data['text'])
y = data['label']

# 2. División train/test
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42, stratify=y)

# 3. Modelo
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# 4. Evaluación
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9644670050761421
              precision    recall  f1-score   support

           0       0.99      0.94      0.97       109
           1       0.94      0.99      0.96        88

    accuracy                           0.96       197
   macro avg       0.96      0.97      0.96       197
weighted avg       0.97      0.96      0.96       197

